In [9]:
# prompt: Kết nối gg drive

from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [1]:
!pip install -q datasets sacrebleu accelerate evaluate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.8/51.8 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 472.7/472.7 kB 26.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.0/104.0 kB 8.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 11.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 55.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 14.5 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 24.4.1 requires pyarrow<15.0.0a0,>=14.0.1, but you have pyarrow 17.0.0 which is incompatible.
ibis-framework 8.0.0 requires pyarrow<16,>=2, but you have pyarrow 17.0.0 which i

In [2]:
import os
import torch
import torch.nn as nn
from transformers import *
import sacrebleu
from datasets import load_dataset
from torch.utils.data import Dataset
import numpy as np
import evaluate

/usr/local/lib/python3.10/dist-packages/transformers/deepspeed.py:24: FutureWarning: transformers.deepspeed module is deprecated and will be removed in a future version. Please import deepspeed modules directly from transformers.integrations
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
GroupViT models are not usable since `tensorflow_probability` can't be loaded. It seems you have `tensorflow_probability` installed with the wrong tensorflow version.Please try to reinstall it following the instructions he

In [11]:
data['translation']

[{'en': 'The science behind a climate headline',
  'vi': 'Khoa học đằng sau một tiêu đề về khí hậu'},
 {'en': 'In 4 minutes , atmospheric chemist Rachel Pike provides a glimpse of the massive scientific effort behind the bold headlines on climate change , with her team -- one of thousands who contributed -- taking a risky flight over the rainforest in pursuit of data on a key molecule .',
  'vi': 'Trong 4 phút , chuyên gia hoá học khí quyển Rachel Pike giới thiệu sơ lược về những nỗ lực khoa học miệt mài đằng sau những tiêu đề táo bạo về biến đổi khí hậu , cùng với đoàn nghiên cứu của mình -- hàng ngàn người đã cống hiến cho dự án này -- một chuyến bay mạo hiểm qua rừng già để tìm kiếm thông tin về một phân tử then chốt .'},
 {'en': "I 'd like to talk to you today about the scale of the scientific effort that goes into making the headlines you see in the paper .",
  'vi': 'Tôi muốn cho các bạn biết về sự to lớn của những nỗ lực khoa học đã góp phần làm nên các dòng tít bạn thường thấy 

In [16]:
a = [sample['en'] for sample in data['translation']]
b = [sample['vi'] for sample in data['translation']]
c = max([len(i) for i in a])
d = max([len(i) for i in b])


2610

In [20]:
d

3198

In [19]:

e = [sample['en'] for sample in data['translation'] if len(sample['en']) == 2610]
e

["Thula Mama , Thula Mama , Thula Mama , Thula Mama . Through the mist of the tears in your eyes on my childhood memory , I know the truth in your smile , I know the truth in your smile , piercing through the gloom of my ignorance . Oh , there is a mama lying down sleeping you 're very ill and your heart crying . Wondering , wondering , wondering , wondering where is this world coming to . Is it right the children have to fend for themselves ? No , no , no , no , no. no . Is it right heaping trouble on an old lady 's head ? So unlucky faceless people . Thula Mama Mama , Thula Mama . Thula Mama Mama . Thula Mama , Thula Mama , Thula Mama Mama , Thula Mama Tomorrow it 's going to be better . Tomorrow it 's going to be better to climb , Mama . Thula Mama , Thula Mama . Am I to break into the song like the blues man or troubadour . And then from long distance in no blues club am I to sing , baby , baby , baby , baby , baby , baby , baby , baby , baby , baby , baby , baby . Should I now sto

# Load dataset

In [3]:
class MMTDataset(Dataset):
  def __init__(self, cfg, data_type = "Train"):
    super().__init__()
    self.cfg = cfg
    self.data_type = data_type
    self.src_texts, self.tgt_texts = self.read_data()
    self.src_input_ids, self.src_attention_mask = self.texts_to_sequences(self.src_texts)
    self.tgt_input_ids, self.tgt_attention_mask, self.labels = self.texts_to_sequences(
        self.tgt_texts,
        is_src=False
    )

  def read_data(self):
    data = load_dataset("Angelectronic/IWSLT15_English_Vietnamese", split="train")
    src_text = [sample[self.cfg.src_lang] for sample in data['translation']]
    tgt_text = [sample[self.cfg.tgt_lang] for sample in data['translation']]
    return src_text, tgt_text

  def texts_to_sequences(self, texts, is_src = True):
    if is_src:
      src_inputs = self.cfg.src_tokenizer(
          texts,
          max_length=self.cfg.src_max_len,
          padding="max_length",
          truncation=True,
          return_tensors="pt",
      )
      return (
          src_inputs.input_ids,
          src_inputs.attention_mask
      )
    else:
      if self.cfg.add_special_token:
        texts = [
            " ".join([
                self.cfg.tgt_tokenizer.bos_token,
                text,
                self.cfg.tgt_tokenizer.eos_token
            ])
            for text in texts
        ]
      tgt_inputs = self.cfg.tgt_tokenizer(
          texts,
          max_length=self.cfg.tgt_max_len,
          padding="max_length",
          truncation=True,
          return_tensors="pt",
      )
      labels = tgt_inputs.input_ids.numpy().tolist()
      labels =[[
          -100 if token_id == self.cfg.tgt_tokenizer.pad_token_id else token_id
          for token_id in label
      ] for label in labels]
      labels = torch.LongTensor(labels)

      return (
          tgt_inputs.input_ids,
          tgt_inputs.attention_mask,
          labels
          )

  def __getitem__(self, index):
    return {
        "input_ids": self.src_input_ids[index],
        "attention_mask": self.src_attention_mask[index],
        "decoder_input_ids": self.tgt_input_ids[index],
        "decoder_attention_mask": self.tgt_attention_mask[index],
        "labels": self.labels[index]
    }
  def __len__(self):
    return np.shape(self.src_input_ids)[0]


# Load tokenizer

In [4]:
def postprocess_text(preds, labels):
    preds = [pred.strip() for pred in preds]
    labels = [[label.strip()] for label in labels]
    return preds, labels
def load_tokenizer(model_name_or_path):
    if 'bert' in model_name_or_path.split('-'):
        return BertTokenizerFast.from_pretrained(model_name_or_path)
    elif 'gpt2' in model_name_or_path.split('-'):
        return GPT2TokenizerFast.from_pretrained(model_name_or_path)
    else:
        return AutoTokenizer.from_pretrained(model_name_or_path)

In [5]:
class Train():
  def __init__(self, cfg):
    self.cfg = cfg
    print("Loading tokenizer...")
    self.get_tokenizer()
    print("Loading dataset...")
    self.train_dataset = MMTDataset(self.cfg, "train")
    self.eval_dataset = MMTDataset(self.cfg, "validation")
    print("Loading model...")
    self.get_model()
    print("Loading Metric...")
    self.bleu_metric = evaluate.load("sacrebleu")

    print("Check Save Model Path")
    if not os.path.exists(self.cfg.ckpt_dir):
        os.mkdir(self.cfg.ckpt_dir)

  def get_tokenizer(self):
    self.cfg.src_tokenizer = load_tokenizer(self.cfg.src_model_name)
    if "bert" in self.cfg.tgt_model_name.split('-'):
      self.cfg.tgt_tokenizer = load_tokenizer(self.cfg.tgt_model_name)
      self.cfg.add_special_token = False
      self.cfg.bos_token_id = self.cfg.tgt_tokenizer.cls_token_id
      self.cfg.eos_token_id = self.cfg.tgt_tokenizer.sep_token_id
      self.cfg.pad_token_id = self.cfg.tgt_tokenizer.pad_token_id
    else:
      self.cfg.tgt_tokenizer = load_tokenizer(self.cfg.tgt_model_name)
      self.cfg.add_special_token = True
      self.cfg.tgt_tokenizer.add_special_tokens(
          {
            "bos_token": "[BOS]",
            "eos_token": "[EOS]",
            "pad_token": "[PAD]"
          }
      )
      self.cfg.bos_token_id = self.cfg.tgt_tokenizer.bos_token_id
      self.cfg.eos_token_id = self.cfg.tgt_tokenizer.eos_token_id
      self.cfg.pad_token_id = self.cfg.tgt_tokenizer.pad_token_id
    self.cfg.tgt_tokenizer.save_pretrained(
            os.path.join(self.cfg.ckpt_dir, f"{self.cfg.src_lang}_tokenizer_{cfg.src_model_name}")
        )
    self.cfg.tgt_tokenizer.save_pretrained(
            os.path.join(self.cfg.ckpt_dir, f"{self.cfg.tgt_lang}_tokenizer_{cfg.tgt_model_name}")
        )



  def get_model(self):
    self.model = EncoderDecoderModel.from_encoder_decoder_pretrained(
        self.cfg.src_model_name,
        self.cfg.tgt_model_name
    )
    self.model.decoder.resize_token_embeddings(
        len(self.cfg.tgt_tokenizer)
    )
    self.model.config.decoder_start_token_id = self.cfg.bos_token_id
    self.model.config.eos_token_id = self.cfg.eos_token_id
    self.model.config.pad_token_id = self.cfg.pad_token_id
    self.model.config.vocab_size = len(self.cfg.tgt_tokenizer)
    self.model.config.max_length = self.cfg.max_length_decoder
    self.model.config.min_length = self.cfg.min_length_decoder
    self.model.config.no_repeat_ngram_size = 3
    self.model.config.early_stopping = True
    self.model.config.length_penalty = 2.0
    self.model.config.num_beams = self.cfg.beam_size

  def train(self):
    print("Training...")
    if self.cfg.use_eval_steps:
        training_args = Seq2SeqTrainingArguments(
            predict_with_generate=True,
            evaluation_strategy="steps",
            save_strategy='steps',
            save_steps=self.cfg.eval_steps,
            eval_steps=self.cfg.eval_steps,
            output_dir=self.cfg.ckpt_dir,
            per_device_train_batch_size=self.cfg.train_batch_size,
            per_device_eval_batch_size=self.cfg.eval_batch_size,
            learning_rate=self.cfg.learning_rate,
            weight_decay=0.005,
            num_train_epochs=self.cfg.num_train_epochs
        )
    else:
        training_args = Seq2SeqTrainingArguments(
            predict_with_generate=True,
            evaluation_strategy="epoch",
            save_strategy='epoch',
            output_dir=self.cfg.ckpt_dir,
            per_device_train_batch_size=self.cfg.train_batch_size,
            per_device_eval_batch_size=self.cfg.eval_batch_size,
            learning_rate=self.cfg.learning_rate,
            weight_decay=0.005,
            num_train_epochs=self.cfg.num_train_epochs
        )
    data_collator = DataCollatorForSeq2Seq(
        self.cfg.tgt_tokenizer,
        model=self.model
    )

    trainer = Seq2SeqTrainer(
        self.model,
        training_args,
        train_dataset=self.train_dataset,
        eval_dataset=self.eval_dataset,
        data_collator=data_collator,
        tokenizer=self.cfg.tgt_tokenizer,
        compute_metrics=self.compute_metrics
    )

    trainer.train()

  def compute_metrics(self, eval_preds):
      preds, labels = eval_preds
      if isinstance(preds, tuple):
          preds = preds[0]
      decoded_preds = self.cfg.tgt_tokenizer.batch_decode(preds, skip_special_tokens=True)

      labels = np.where(labels != -100, labels, self.cfg.tgt_tokenizer.pad_token_id)
      decoded_labels = self.cfg.tgt_tokenizer.batch_decode(labels, skip_special_tokens=True)

      decoded_preds, decoded_labels = postprocess_text(decoded_preds, decoded_labels)

      result = self.bleu_metric.compute(
          predictions=decoded_preds,
          references=decoded_labels
      )

      result = {"bleu_score": result["score"]}

      prediction_lens = [np.count_nonzero(pred != self.cfg.tgt_tokenizer.pad_token_id) for pred in preds]
      result["gen_len"] = np.mean(prediction_lens)
      result = {k: round(v, 4) for k, v in result.items()}

      return result


In [6]:
class BaseConfig:
    def __init__(self, **kwargs):
        for k, v in kwargs.items():
            setattr(self, k, v)

class NMTConfig(BaseConfig):
    src_lang = 'en'
    tgt_lang = 'vi'
    src_max_len = 100
    tgt_max_len = 100

    src_model_name = "bert-base-multilingual-cased"
    tgt_model_name = "bert-base-multilingual-cased"

    device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
    learning_rate = 3e-5
    train_batch_size = 32
    eval_batch_size = 32
    num_train_epochs = 5
    ckpt_dir = src_model_name + '_to_' + tgt_model_name
    use_eval_steps = True
    eval_steps = 5000

    max_length_decoder = 100
    min_length_decoder = 25
    beam_size = 3

cfg = NMTConfig()

In [7]:
manager = Train(cfg)

Loading tokenizer...


tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/996k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.96M [00:00<?, ?B/s]

loading file vocab.txt from cache at /root/.cache/huggingface/hub/models--bert-base-multilingual-cased/snapshots/3f076fdb1ab68d5b2880cb87a0886f315b8146f8/vocab.txt
loading file tokenizer.json from cache at /root/.cache/huggingface/hub/models--bert-base-multilingual-cased/snapshots/3f076fdb1ab68d5b2880cb87a0886f315b8146f8/tokenizer.json
loading file added_tokens.json from cache at None
loading file special_tokens_map.json from cache at None
loading file tokenizer_config.json from cache at /root/.cache/huggingface/hub/models--bert-base-multilingual-cased/snapshots/3f076fdb1ab68d5b2880cb87a0886f315b8146f8/tokenizer_config.json


...

loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--bert-base-multilingual-cased/snapshots/3f076fdb1ab68d5b2880cb87a0886f315b8146f8/config.json
Model config BertConfig {
  "_name_or_path": "bert-base-multilingual-cased",
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "directionality": "bidi",
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "pooler_fc_size": 768,
  "pooler_num_attention_heads": 12,
  "pooler_num_fc_layers": 3,
  "pooler_size_per_head": 128,
  "pooler_type": "first_token_transform",
  "position_embedding_type": "absolute",
  "transformers_version": "4.44.2",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size":

model.safetensors:   0%|          | 0.00/714M [00:00<?, ?B/s]

loading weights file model.safetensors from cache at /root/.cache/huggingface/hub/models--bert-base-multilingual-cased/snapshots/3f076fdb1ab68d5b2880cb87a0886f315b8146f8/model.safetensors
A pretrained model of type `BertModel` contains parameters that have been renamed internally (a few are listed below but more are present in the model):
* `bert.embeddings.LayerNorm.gamma` -> `bert.embeddings.LayerNorm.weight`
* `bert.encoder.layer.0.attention.output.LayerNorm.gamma` -> `{'bert.embeddings.LayerNorm.gamma': 'bert.embeddings.LayerNorm.weight', 'bert.encoder.layer.0.attention.output.LayerNorm.gamma': {...}, 'bert.encoder.layer.0.output.LayerNorm.gamma': {...}, 'bert.encoder.layer.1.attention.output.LayerNorm.gamma': {...}, 'bert.encoder.layer.1.output.LayerNorm.gamma': {...}, 'bert.encoder.layer.10.attention.output.LayerNorm.gamma': {...}, 'bert.encoder.layer.10.output.LayerNorm.gamma': {...}, 'bert.encoder.layer.11.attention.output.LayerNorm.gamma': {...}, 'bert.encoder.layer.11.output.

Loading Metric...


Check Save Model Path


In [8]:
manager.train()

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


Training...


***** Running training *****
  Num examples = 133,166
  Num Epochs = 5
  Instantaneous batch size per device = 32
  Total train batch size (w. parallel, distributed & accumulation) = 32
  Gradient Accumulation steps = 1
  Total optimization steps = 20,810
  Number of trainable parameters = 384,194,811
/usr/local/lib/python3.10/dist-packages/transformers/data/data_collator.py:656: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:278.)
  batch["labels"] = torch.tensor(batch["labels"], dtype=torch.int64)
/usr/local/lib/python3.10/dist-packages/transformers/models/encoder_decoder/modeling_encoder_decoder.py:643: FutureWarning: Version v4.12.0 introduces a better way to train encoder-decoder models by computing the loss inside the encoder-decoder framework rather than in the decoder itself. 

Step,Training Loss,Validation Loss,Bleu Score,Gen Len
5000,2.087800,1.856562,20.323700,34.015400



***** Running Evaluation *****
  Num examples = 133166
  Batch size = 32
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:615: UserWarning: `num_beams` is set to 1. However, `early_stopping` is set to `True` -- this flag is only used in beam-based generation modes. You should set `num_beams>1` or unset `early_stopping`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:634: UserWarning: `num_beams` is set to 1. However, `length_penalty` is set to `2.0` -- this flag is only used in beam-based generation modes. You should set `num_beams>1` or unset `length_penalty`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(

ValueError: The generation config instance is invalid -- `.validate()` throws warnings and/or exceptions. Fix these issues to save the configuration.

Thrown during validation:
[UserWarning('`num_beams` is set to 1. However, `early_stopping` is set to `True` -- this flag is only used in beam-based generation modes. You should set `num_beams>1` or unset `early_stopping`.'), UserWarning('`num_beams` is set to 1. However, `length_penalty` is set to `2.0` -- this flag is only used in beam-based generation modes. You should set `num_beams>1` or unset `length_penalty`.')]

# Test and save model

In [27]:
from transformers import GenerationConfig

# Define a custom generation configuration
generation_config = GenerationConfig(
    num_beams=3,
    early_stopping=True,
    length_penalty=2.0
)

# Assign the custom configuration to the model
manager.model.generation_config = generation_config

# Save the model with the updated configuration
manager.model.save_pretrained("/content/drive/MyDrive/Code/NLP/Machine Translate/Model/")


Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'early_stopping': True, 'num_beams': 3, 'length_penalty': 2.0}
Configuration saved in /content/drive/MyDrive/Code/NLP/Machine Translate/Model/config.json
Configuration saved in /content/drive/MyDrive/Code/NLP/Machine Translate/Model/generation_config.json
Model weights saved in /content/drive/MyDrive/Code/NLP/Machine Translate/Model/model.safetensors


In [28]:
link_model = "/content/drive/MyDrive/Code/NLP/Machine Translate/Model"
link_en_tokenizer = "/content/bert-base-multilingual-cased_to_bert-base-multilingual-cased/en_tokenizer_bert-base-multilingual-cased"
link_vi_tokenizer = "/content/bert-base-multilingual-cased_to_bert-base-multilingual-cased/vi_tokenizer_bert-base-multilingual-cased"
model_1 = EncoderDecoderModel.from_pretrained(link_model)
en_tokenizer = BertTokenizer.from_pretrained(link_en_tokenizer)
vi_tokenizer = BertTokenizer.from_pretrained(link_vi_tokenizer)

loading configuration file /content/drive/MyDrive/Code/NLP/Machine Translate/Model/config.json
Model config EncoderDecoderConfig {
  "architectures": [
    "EncoderDecoderModel"
  ],
  "decoder": {
    "_name_or_path": "bert-base-multilingual-cased",
    "add_cross_attention": true,
    "architectures": [
      "BertForMaskedLM"
    ],
    "attention_probs_dropout_prob": 0.1,
    "bad_words_ids": null,
    "begin_suppress_tokens": null,
    "bos_token_id": null,
    "chunk_size_feed_forward": 0,
    "classifier_dropout": null,
    "cross_attention_hidden_size": null,
    "decoder_start_token_id": null,
    "directionality": "bidi",
    "diversity_penalty": 0.0,
    "do_sample": false,
    "early_stopping": false,
    "encoder_no_repeat_ngram_size": 0,
    "eos_token_id": null,
    "exponential_decay_length_penalty": null,
    "finetuning_task": null,
    "forced_bos_token_id": null,
    "forced_eos_token_id": null,
    "hidden_act": "gelu",
    "hidden_dropout_prob": 0.1,
    "hidden_s

In [35]:
sentence = "Hello, how are you?"

inputs = en_tokenizer(sentence, return_tensors="pt")

with torch.no_grad():
    outputs = model_1.generate(
        inputs["input_ids"],
        max_length=40,
        num_beams=3,
        early_stopping=True,
        decoder_start_token_id=model_1.config.decoder_start_token_id
    )
    translated_sentence = vi_tokenizer.decode(outputs[0], skip_special_tokens=True)

print("Câu gốc:", sentence)
print("Câu dịch:", translated_sentence)

Câu gốc: Hello, how are you?
Câu dịch: Xin chào, cô ấy như thế nào? Xin cháu như thế nào? Xin cháu như thế nào? Xin như thế nào?


In [36]:
# Đường dẫn lưu cho các tokenizer
en_tokenizer_save_path = "/content/drive/MyDrive/Code/NLP/Machine Translate/tokenizer_en"
vi_tokenizer_save_path = "/content/drive/MyDrive/Code/NLP/Machine Translate/tokenizer_vi"

# Lưu tokenizer tiếng Anh
en_tokenizer.save_pretrained(en_tokenizer_save_path)

# Lưu tokenizer tiếng Việt
vi_tokenizer.save_pretrained(vi_tokenizer_save_path)


tokenizer config file saved in /content/drive/MyDrive/Code/NLP/Machine Translate/tokenizer_en/tokenizer_config.json
Special tokens file saved in /content/drive/MyDrive/Code/NLP/Machine Translate/tokenizer_en/special_tokens_map.json
tokenizer config file saved in /content/drive/MyDrive/Code/NLP/Machine Translate/tokenizer_vi/tokenizer_config.json
Special tokens file saved in /content/drive/MyDrive/Code/NLP/Machine Translate/tokenizer_vi/special_tokens_map.json


('/content/drive/MyDrive/Code/NLP/Machine Translate/tokenizer_vi/tokenizer_config.json',
 '/content/drive/MyDrive/Code/NLP/Machine Translate/tokenizer_vi/special_tokens_map.json',
 '/content/drive/MyDrive/Code/NLP/Machine Translate/tokenizer_vi/vocab.txt',
 '/content/drive/MyDrive/Code/NLP/Machine Translate/tokenizer_vi/added_tokens.json')